In [9]:
import torch
import torch.nn as nn
import numpy as np

L, W = 1.0, 0.5  # Length and width of the domain (in cm)
lambda_ = 5.0e9  # Elastic constant (Pa)
mu = 5.0e9       # Shear modulus (Pa)
h = 1.0          # Thickness (cm)

class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 128),
            nn.Softplus(beta=10),
            nn.Linear(128, 128),
            nn.Softplus(beta=10),
            nn.Linear(128, 128),
            nn.Softplus(beta=10),
            nn.Linear(128, 2)
        )

    def forward(self, x, y):
        inputs = torch.cat([x, y], dim=1)
        raw = self.net(inputs)

        enforced_out = torch.zeros_like(raw)
        enforced_out[:, 0] = (x.flatten() + 0.5) * raw[:, 0]  # u_x
        enforced_out[:, 1] = (x.flatten() + 0.5) * raw[:, 1]  # u_y

        corner_mask = (x[:, 0] == 0.5)
        enforced_out[corner_mask, 1] = 0.0

        return enforced_out

class SelfAdaptiveWeights(nn.Module):
    def __init__(self):
        super(SelfAdaptiveWeights, self).__init__()
        # Trainable log-weights for PDE loss and 4 boundary conditions
        self.log_w_pde = nn.Parameter(torch.tensor(0.0))
        self.log_w_bcA = nn.Parameter(torch.tensor(0.0))
        self.log_w_bcD = nn.Parameter(torch.tensor(0.0))
        self.log_w_bcB = nn.Parameter(torch.tensor(0.0))
        self.log_w_bcC = nn.Parameter(torch.tensor(0.0))

    def get_weights(self):
        """Return softmax-normalized weights."""
        weights = torch.exp(torch.stack([
            self.log_w_pde, self.log_w_bcA, self.log_w_bcD, self.log_w_bcB, self.log_w_bcC
        ]))
        return weights / weights.sum()

def strain_tensor(u_x, u_y, x, y):
    u_x_x = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]
    u_y_y = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True)[0]
    u_x_y = torch.autograd.grad(u_x, y, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]
    u_y_x = torch.autograd.grad(u_y, x, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True)[0]

    return u_x_x, u_y_y, 0.5 * (u_x_y + u_y_x)

def stress_tensor(E_xx, E_yy, E_xy):
    scale_factor = 1e9
    trace_E = E_xx + E_yy
    sigma_xx = h * ((lambda_/scale_factor)*trace_E + 2*(mu/scale_factor)*E_xx)
    sigma_yy = h * ((lambda_/scale_factor)*trace_E + 2*(mu/scale_factor)*E_yy)
    sigma_xy = h * (2*(mu/scale_factor)*E_xy)
    return sigma_xx, sigma_yy, sigma_xy

def physics_loss(model, x, y):
    x.requires_grad_(True)
    y.requires_grad_(True)
    u = model(x, y)
    u_x, u_y = u[:, 0:1], u[:, 1:2]

    E_xx, E_yy, E_xy = strain_tensor(u_x, u_y, x, y)
    sigma_xx, sigma_yy, sigma_xy = stress_tensor(E_xx, E_yy, E_xy)

    sigma_xx_x = torch.autograd.grad(sigma_xx, x, grad_outputs=torch.ones_like(sigma_xx), retain_graph=True, create_graph=True)[0]
    sigma_xy_y = torch.autograd.grad(sigma_xy, y, grad_outputs=torch.ones_like(sigma_xy), retain_graph=True, create_graph=True)[0]
    sigma_yy_y = torch.autograd.grad(sigma_yy, y, grad_outputs=torch.ones_like(sigma_yy), retain_graph=True, create_graph=True)[0]
    sigma_xy_x = torch.autograd.grad(sigma_xy, x, grad_outputs=torch.ones_like(sigma_xy), retain_graph=True, create_graph=True)[0]

    return torch.mean((sigma_xx_x + sigma_xy_y) ** 2 + (sigma_yy_y + sigma_xy_x) ** 2)

def boundary_condition_loss(model, L, W):
    def bc_loss(x, y, expected_u):
        u = model(x, y)
        return torch.mean((u - expected_u) ** 2)

    # Boundary A: u_x = 0, u_y = 0 at x = -L/2
    x_A = -L / 2 * torch.ones(100, 1, requires_grad=True)
    y_A = torch.linspace(-W / 2, W / 2, 100).reshape(-1, 1).requires_grad_()
    loss_A = bc_loss(x_A, y_A, torch.zeros_like(y_A))

    # Boundary D: u_x = 0.025 * L, u_y = 0 at x = L/2
    x_D = L / 2 * torch.ones(100, 1, requires_grad=True)
    y_D = torch.linspace(-W / 2, W / 2, 100).reshape(-1, 1).requires_grad_()
    loss_D = bc_loss(x_D, y_D, torch.tensor([0.025 * L, 0.0]))

    # Boundary C (y=0): Only force u_y=0
    x_C = torch.linspace(-L/2, L/2, 200).reshape(-1, 1).requires_grad_()
    y_C = torch.zeros_like(x_C, requires_grad=True)
    u_C = model(x_C, y_C)
    loss_C = torch.mean(u_C[:, 1] ** 2)

    # Boundary B: traction-free (σ_xx = σ_xy = 0) at y = W/2
    x_B = torch.linspace(-L / 2, L / 2, 100).reshape(-1, 1).requires_grad_()
    y_B = W / 2 * torch.ones_like(x_B, requires_grad=True)
    u_B_x, u_B_y = model(x_B, y_B)[:, 0:1], model(x_B, y_B)[:, 1:2]
    E_xx_B, E_yy_B, E_xy_B = strain_tensor(u_B_x, u_B_y, x_B, y_B)
    sigma_xx_B, sigma_yy_B, sigma_xy_B = stress_tensor(E_xx_B, E_yy_B, E_xy_B)
    loss_B = torch.mean(sigma_yy_B**2 + sigma_xy_B**2)

    return loss_A, loss_D, loss_B, loss_C

def train_pinn(model, weight_model, optimizer, weight_optimizer, n_epochs, n_points, L, W):
    for epoch in range(n_epochs):
        loss_pde = physics_loss(model, torch.rand((n_points, 1)), torch.rand((n_points, 1)))
        loss_A, loss_D, loss_B, loss_C = boundary_condition_loss(model, L, W)

        w_pde, w_A, w_D, w_B, w_C = weight_model.get_weights()
        loss = w_pde * loss_pde + w_A * loss_A + w_D * loss_D + w_B * loss_B + w_C * loss_C

        optimizer.zero_grad()
        weight_optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        weight_optimizer.step()

        if (epoch % 500 == 0):
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}")
            print(f"    Weights:")
            print(f"        PDE Residual:        {w_pde.item():.3f}")
            print(f"        BC A (x=-L/2):       {w_A.item():.3f}")
            print(f"        BC D (x=L/2):        {w_D.item():.3f}")
            print(f"        BC B (y=W/2):        {w_B.item():.3f}")
            print(f"        BC C (y=0):          {w_C.item():.3f}")

model = PINN()
weight_model = SelfAdaptiveWeights()
train_pinn(model, weight_model, torch.optim.Adam(model.parameters(), lr=0.001), torch.optim.Adam(weight_model.parameters(), lr=0.01), 10000, 1000, L, W)


Epoch 0, Loss: 0.080952
    Weights:
        PDE Residual:        0.200
        BC A (x=-L/2):       0.200
        BC D (x=L/2):        0.200
        BC B (y=W/2):        0.200
        BC C (y=0):          0.200
Epoch 500, Loss: 0.000032
    Weights:
        PDE Residual:        0.155
        BC A (x=-L/2):       0.218
        BC D (x=L/2):        0.210
        BC B (y=W/2):        0.199
        BC C (y=0):          0.218
Epoch 1000, Loss: 0.000025
    Weights:
        PDE Residual:        0.155
        BC A (x=-L/2):       0.221
        BC D (x=L/2):        0.207
        BC B (y=W/2):        0.196
        BC C (y=0):          0.221
Epoch 1500, Loss: 0.000021
    Weights:
        PDE Residual:        0.155
        BC A (x=-L/2):       0.225
        BC D (x=L/2):        0.205
        BC B (y=W/2):        0.189
        BC C (y=0):          0.225
Epoch 2000, Loss: 0.000015
    Weights:
        PDE Residual:        0.156
        BC A (x=-L/2):       0.230
        BC D (x=L/2):        0.203

In [10]:
import pandas as pd

file_path = '/Users/murat/Downloads/data.csv'
comparison_data = pd.read_csv(file_path)

x_vals = torch.tensor(comparison_data['X'].values, dtype=torch.float32).reshape(-1,1)
y_vals = torch.tensor(comparison_data['Y'].values, dtype=torch.float32).reshape(-1,1)
u_x_actual = comparison_data['u_x_actual'].values
u_y_actual = comparison_data['u_y_actual'].values


y_abs = y_vals.abs()

model.eval()
with torch.no_grad():
    u_pred_top = model(x_vals, y_abs)


u_pred = u_pred_top.clone()
mask_bottom = (y_vals > 0).view(-1)
u_pred[mask_bottom, 1] *= -1.0

u_x_pred = u_pred[:,0].numpy()
u_y_pred = u_pred[:,1].numpy()

comparison_data['u_x_pred'] = u_x_pred
comparison_data['u_y_pred'] = u_y_pred


comparison_data['error_u_x'] = abs(u_x_actual - u_x_pred)
comparison_data['error_u_y'] = abs(u_y_actual - u_y_pred)

def safe_percent_error(a, p):
    if np.isclose(a, 0.0):
        return 0.0
    else:
        return abs(a - p) / abs(a) * 100.0

comparison_data['percent_error_u_x'] = [
    safe_percent_error(a, p) for a, p in zip(u_x_actual, u_x_pred)
]
comparison_data['percent_error_u_y'] = [
    safe_percent_error(a, p) for a, p in zip(u_y_actual, u_y_pred)
]

print(comparison_data)

avg_percent_error_x = comparison_data['percent_error_u_x'].mean()
avg_percent_error_y = comparison_data['percent_error_u_y'].mean()

print(f"Average Percent Error for u_x: {avg_percent_error_x:.2f}%")
print(f"Average Percent Error for u_y: {avg_percent_error_y:.2f}%")


         X     Y    u_x_actual    u_y_actual  u_x_pred  u_y_pred  \
0    -0.50 -0.25  1.483633e-19  1.263433e-19 -0.000000 -0.000000   
1    -0.48 -0.25  9.101041e-04  9.363998e-04  0.000151 -0.000050   
2    -0.50 -0.23 -1.530000e-20 -4.190000e-20 -0.000000  0.000000   
3    -0.48 -0.23  5.332043e-04  6.506182e-04  0.000054  0.000045   
4    -0.50 -0.21  2.321675e-21 -6.630000e-21 -0.000000  0.000000   
...    ...   ...           ...           ...       ...       ...   
1321  0.48  0.23  2.446680e-02 -6.510000e-04  0.022458 -0.000081   
1322  0.50  0.21  2.500000e-02  3.680167e-18  0.022531 -0.000000   
1323  0.48  0.25  2.408990e-02 -9.360000e-04  0.022447 -0.000064   
1324  0.50  0.23  2.500000e-02  1.768693e-18  0.022508 -0.000000   
1325  0.50  0.25  2.500000e-02  2.080774e-17  0.022503 -0.000000   

         error_u_x     error_u_y  percent_error_u_x  percent_error_u_y  
0     1.483633e-19  1.263433e-19           0.000000           0.000000  
1     7.594807e-04  9.863700e-04     